In [1]:
!pip install openeo geopandas shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.8/337.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.5/208.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.4 MB/s eta 0:00:00
  Attempting uninstall: xarray
    Found existing installation: xarray 2025.12.0
    Uninstalling xarray-2025.12.0:
      Successfully uninstalled xarray-2025.12.0


In [2]:
import openeo

# Verbinden met de openEO Platform backend
conn = openeo.connect("openeo.cloud").authenticate_oidc()

print("✅ Verbonden met openEO:", conn)

Visit https://aai.egi.eu/device?user_code=XKBE-VNGO 📋 to authenticate.

✅ Authorized successfully

Authenticated using device code flow.
✅ Verbonden met openEO: <Connection to 'https://openeocloud.vito.be/openeo/1.0.0/' with OidcBearerAuth>


In [3]:
from google.colab import files

# ⬆️ Upload je KML-bestand (selecteer het vanaf je computer)
uploaded = files.upload()

Saving Lansingerland.kml to Lansingerland.kml


In [4]:
import geopandas as gpd

# 🔧 Zet hier exact de naam van je KML-bestand
kml_path = "/content/Lansingerland.kml"

# 📥 KML inlezen
gdf = gpd.read_file(kml_path, driver="KML")

# 🔍 Controle: wat zit erin?
print("Aantal features:", len(gdf))
print("Kolommen:", gdf.columns)

# 🗺️ Toon de geometrie (gemeentegrens)
gdf

Aantal features: 1
Kolommen: Index(['id', 'Name', 'description', 'timestamp', 'begin', 'end',
       'altitudeMode', 'tessellate', 'extrude', 'visibility', 'drawOrder',
       'icon', 'snippet', 'geometry'],
      dtype='object')


,id,Name,description,timestamp,begin,end,altitudeMode,tessellate,extrude,visibility,drawOrder,icon,snippet,geometry
0,ID_00000,GM1621,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",NaT,NaT,NaT,clampToGround,-1,0,-1,NaN,None,,"MULTIPOLYGON Z (((4.48356 51.97433 0, 4.48364 ..."


In [5]:
# 📐 Zet geometrie om naar GeoJSON
gdf = gdf.to_crs("EPSG:4326")  # Zorg dat het in WGS84 staat
geometry = gdf.geometry.iloc[0]  # Neem de eerste (en enige) geometrie

geojson_geometry = geometry.__geo_interface__  # openEO-ready
print("✅ GeoJSON polygoon klaar voor gebruik in openEO.")

✅ GeoJSON polygoon klaar voor gebruik in openEO.


In [6]:
# 📦 Ophalen van Sentinel-2 data voor Lansingerland
cube = (
    conn.load_collection(
        "SENTINEL2_L2A",
        spatial_extent=geojson_geometry,
        temporal_extent=["2025-01-01", "2025-12-31"],
        bands=["B03", "B08"],  # Green + NIR
    )
)

print("✅ Sentinel-2 data cube aangemaakt.")

✅ Sentinel-2 data cube aangemaakt.


In [7]:
from openeo.processes import subtract, add, divide

green = cube.band("B03")
nir = cube.band("B08")

# 💧 NDWI = (Green - NIR) / (Green + NIR)
numerator = subtract(green, nir)
denominator = add(green, nir)
ndwi_cube = divide(numerator, denominator)

print("✅ NDWI handmatig opgebouwd met expliciete operaties.")

✅ NDWI handmatig opgebouwd met expliciete operaties.


In [8]:
# 📦 Binary watermasker per tijdstap maken op basis van NDWI
# Water = 1 als NDWI > 0.2

water_mask_cube = ndwi_cube > 0.2

print("✅ Watermaskers per tijdstap zijn aangemaakt (0 = geen water, 1 = water)")

✅ Watermaskers per tijdstap zijn aangemaakt (0 = geen water, 1 = water)


✅ Is een nieuw raster nodig?

Ja, maar we hoeven dat niet handmatig te bouwen.
We reduceren de tijdsdimensie met een sum-operatie:

In [9]:
from openeo.processes import gt, int as p_int

# 🧠 Vergelijk NDWI direct met threshold, expliciet als openEO expressie
water_mask_cube = gt(ndwi_cube, 0.2)

# 🔧 Zet booleans om naar integers (1 = water, 0 = geen water)
water_mask_int = p_int(water_mask_cube)

print("✅ Boolean → integer conversie opnieuw en cleaner uitgevoerd.")

✅ Boolean → integer conversie opnieuw en cleaner uitgevoerd.


In [10]:
# 📊 Tellen hoe vaak water werd gedetecteerd per pixel in 2025
water_count = water_mask_int.reduce_dimension(
    dimension="t",  # tijdsdimensie
    reducer="sum"   # tel het aantal keren '1' = water
)

print("✅ Waterfrequentie per pixel is berekend.")

✅ Waterfrequentie per pixel is berekend.


In [11]:
# ✅ Voeg expliciete save toe (GeoTIFF)
export_cube = water_count.save_result(format="GTiff")

# ✅ Start veilige batch job zonder .start_and_wait()
job = conn.create_job(export_cube)
job.start()

print("🚀 Job gestart met ID:", job.job_id)
print("⏳ Even wachten tot job klaar is...")
job.wait_for_completion()

# ✅ Downloaden als alles goed is
job.download_results("results/")
print("✅ GeoTIFF succesvol opgeslagen in: results/")

OpenEoApiError: [500] Internal: Unexpected error during 'subtract': Exception('Datatype unknown.'). The process had these arguments: {'x': DryRunDataCube(metadata=CollectionMetadata(dimension_names=['x', 'y', 't'])), 'y': DryRunDataCube(metadata=CollectionMetadata(dimension_names=['x', 'y', 't']))}  (Upstream ref: 'r-26012613011549dbb24fbd807c0d1173') (ref: r-2601261301114e6e936c8271e51e1536)

In [ ]:
from openeo.processes import subtract, add, divide, gt, int as p_int

# 1. Bands ophalen
green = cube.band("B03")
nir = cube.band("B08")

# 2. NDWI bouwen (handmatig, zodat we types afdwingen)
ndwi = divide(subtract(green, nir), add(green, nir))

# 3. Watermask: NDWI > 0.2
mask = gt(ndwi, 0.2)

# 4. True/False → 1/0
mask_int = p_int(mask)

# 5. Tel het aantal keren '1' over tijdsdimensie
water_count = mask_int.reduce_dimension(dimension="t", reducer="sum")

# 6. Opslaan als GeoTIFF
result = water_count.save_result(format="GTiff")

# 7. Nieuwe job maken en uitvoeren
job = conn.create_job(result)
job.start()
job.wait_for_completion()
job.download_results("results/")

print("✅ Alles gelukt! GeoTIFF opgeslagen in: results/")

OpenEoApiError: [500] Internal: Unexpected error during 'subtract': Exception('Datatype unknown.'). The process had these arguments: {'x': DryRunDataCube(metadata=CollectionMetadata(dimension_names=['x', 'y', 't'])), 'y': DryRunDataCube(metadata=CollectionMetadata(dimension_names=['x', 'y', 't']))}  (Upstream ref: 'r-260120153055427b8ecb6261554a0021') (ref: r-26012015305149acb74b98eeb23ac034)